In [1]:
print('IT IS RUNNING!')
"""
What this script does (UPDATED):
1. Load MULTIPLE CSVs with patient events (data_0.csv ... data_4.csv), tag each row with its file of origin (source_file),
   and combine into one dataframe. 
2. For each patient:
   - Find clusters of activity days where several readings are taken in conjunction with each other.
   - Build index days outside clusters after a sparse gap.
   - Compute basic features (last value, days since last, MEDIAN, IQR, count) for 4 codes over lookback window. 
   - Label: cluster start between days (EXCLUSION_BEFORE_CLUSTER_DAYS+1)..PREDICTION_HORIZON_DAYS after index (default day 8..90).
3. Combine all indices into one dataset.
4. Train/evaluate Random Forest using 5-fold "leave-one-CSV-out" CV (train 5 times),
   BUT only print ONE final combined set of metrics using pooled out-of-fold predictions (no per-fold AUROC prints).
"""

import argparse
from pathlib import Path
import sys
import math

import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, average_precision_score, brier_score_loss, classification_report
from sklearn.impute import SimpleImputer



# Default input CSVs (expects the same schema in each file)
# NOTE: These are absolute paths on your machine; change them if you move the project.
DEFAULT_CSV_PATHS = ["/Users/colleenohare/Desktop/Bioinformatics_MSC/RETFOUND/Chris_Sainsbury/Inspect_Dataset/inspect_data_csv/data_0.csv",
                     "/Users/colleenohare/Desktop/Bioinformatics_MSC/RETFOUND/Chris_Sainsbury/Inspect_Dataset/inspect_data_csv/data_1.csv",
                     "/Users/colleenohare/Desktop/Bioinformatics_MSC/RETFOUND/Chris_Sainsbury/Inspect_Dataset/inspect_data_csv/data_2.csv",
                     "/Users/colleenohare/Desktop/Bioinformatics_MSC/RETFOUND/Chris_Sainsbury/Inspect_Dataset/inspect_data_csv/data_3.csv",
                     "/Users/colleenohare/Desktop/Bioinformatics_MSC/RETFOUND/Chris_Sainsbury/Inspect_Dataset/inspect_data_csv/data_4.csv"]

# Kept for compatibility with older single-split versions of the script.
# This CV script does NOT use a single global train/test split.  
DEFAULT_TEST_SIZE = 0.2  # kept for compatibility (not used as a single split in this CV script) 

# How far back (in days) we look to compute features for each index timestamp
DEFAULT_LOOKBACK_DAYS = 90

# Cluster logic:
# - sparse_gap_days: minimum quiet gap before a run can be considered a "new" cluster
# - dense_gap_days: max gap allowed between consecutive observation days inside a cluster
# - min_cluster_days: minimum number of activity days required to call it a cluster
DEFAULT_SPARSE_GAP_DAYS = 6
DEFAULT_DENSE_GAP_DAYS = 2
DEFAULT_MIN_CLUSTER_DAYS = 4

# Restrict cluster discovery to a specific "table" value if present (commonly "measurement")
DEFAULT_CLUSTER_FROM_TABLE = "measurement"

# Label window:
# Exclude clusters that start too soon after the index day (buffer to avoid leakage / immediate activity)
EXCLUSION_BEFORE_CLUSTER_DAYS = 7
# How far after index we look for the start of a future cluster (positive label horizon)
PREDICTION_HORIZON_DAYS = 90

# Clinical codes used as features (4-variable toy set)
CODE_SBP = "LOINC/8480-6"          # systolic BP
CODE_HR = "LOINC/8867-4"           # heart rate
CODE_GLUCOSE = "SNOMED/271649006"  # glucose levels
CODE_HBA1C = "SNOMED/271650006"    # HbA1c levels

# Expected columns (documentation only — script does not hard-enforce this list)
EXPECTED_COLUMNS = [
    "subject_id", "time", "code", "numeric_value", "care_site_id", "clarity_table",
    "end", "note_id", "provider_id", "table", "text_value", "unit", "visit_id"
]



# Parses mixed-format datetime strings in a pandas Series into datetimes.
# - Handles "dd/mm/YYYY HH:MM" style with explicit format
# - Falls back to pandas parser for other formats
def parse_mixed_datetime(series):
    series = series.astype("string")
    mask_slash = series.str.contains("/", na=False)
    parsed_slash = pd.to_datetime(series.where(mask_slash), format="%d/%m/%Y %H:%M", errors="coerce")
    parsed_other = pd.to_datetime(series.where(~mask_slash), errors="coerce", dayfirst=False)
    return parsed_slash.fillna(parsed_other)


# Loads a CSV, cleans and parses time and numeric fields, and sorts records by subject and time.
# - Parses the "time" column (mixed formats supported)
# - Coerces numeric_value to float (invalid values become NaN)
# - Stable sort by subject_id then time for reproducibility
def load_csv_simple(path):
    print("Loading CSV from", path)
    df = pd.read_csv(path, sep=",", encoding="utf-8-sig", low_memory=False)
    df["time"] = parse_mixed_datetime(df["time"])
    df["numeric_value"] = pd.to_numeric(df.get("numeric_value"), errors="coerce")
    df = df.sort_values(["subject_id", "time"], kind="mergesort").reset_index(drop=True)
    return df


# Identifies clusters of consecutive days with observations for a patient based on gap and minimum-length rules.
# Cluster definition (day-level):
# - collapse events to unique calendar days
# - a "run" is consecutive activity days where gaps between adjacent days are <= dense_gap_days
# - a run becomes a "cluster" if:
#    (1) run_length >= min_cluster_days
#    (2) the run is preceded by a sufficiently sparse gap (>= sparse_gap_days) OR it's the first run
# Optional: restrict cluster-finding to a chosen table (e.g., measurement), falling back to all if empty.
def find_clusters_for_patient(patient_df, sparse_gap_days, dense_gap_days, min_cluster_days, cluster_from_table):
    # Optionally restrict clustering to a specific source table; fall back if empty
    if cluster_from_table is not None and "table" in patient_df.columns:
        chosen = patient_df[(patient_df["table"] == cluster_from_table) & (~patient_df["time"].isna())].copy()
        if chosen.empty:
            chosen = patient_df.dropna(subset=["time"]).copy()
    else:
        # Use all non-null time observations
        chosen = patient_df.dropna(subset=["time"]).copy()

    # No valid timestamps → no clusters
    if chosen.empty:
        return []

    # Collapse timestamps to unique calendar days (midnight-normalized)
    chosen["day"] = chosen["time"].dt.normalize()
    unique_days = sorted(chosen["day"].unique().tolist())

    clusters = []

    # Not enough days to form a cluster
    if len(unique_days) < min_cluster_days:
        return clusters

    # Compute gaps (in days) between consecutive observation days
    day_diffs = []
    for i in range(1, len(unique_days)):
        gap_days = (unique_days[i] - unique_days[i - 1]).days
        day_diffs.append(gap_days)

    # Scan for dense runs separated by gaps larger than dense_gap_days
    run_start_index = 0
    for i, gap in enumerate(day_diffs, start=1):
        if gap <= dense_gap_days:
            continue

        # End current run when gap exceeds dense threshold
        run_end_index = i - 1
        run_length = (run_end_index - run_start_index + 1)

        # Accept run only if long enough and preceded by a sufficiently sparse gap
        if run_length >= min_cluster_days:
            preceding_gap = math.inf if run_start_index == 0 else day_diffs[run_start_index - 1]
            if preceding_gap >= sparse_gap_days:
                start_day = unique_days[run_start_index]
                end_day = unique_days[run_end_index]
                clusters.append((start_day.normalize(), end_day.normalize(), run_length))

        run_start_index = i

    # Handle final run after loop
    run_end_index = len(unique_days) - 1
    run_length = (run_end_index - run_start_index + 1)
    if run_length >= min_cluster_days:
        preceding_gap = math.inf if run_start_index == 0 else day_diffs[run_start_index - 1]
        if preceding_gap >= sparse_gap_days:
            start_day = unique_days[run_start_index]
            end_day = unique_days[run_end_index]
            clusters.append((start_day.normalize(), end_day.normalize(), run_length))

    # Return list of (cluster_start_day, cluster_end_day, number_of_days)
    return clusters


# Builds “index” timestamps for a patient:
# - consider unique observed calendar days
# - choose days that are NOT in any cluster
# - require a sufficiently long quiet gap since the previous observed day (>= sparse_gap_days)
# Output rows are day-level indices, but use the earliest timestamp on that day as index_time.
#
# NEW vs older versions:
# - now also returns the source_file for that index day so we can do leave-one-CSV-out CV.  
def build_index_days(patient_df, clusters, sparse_gap_days):
    # Keep only rows with valid times; if none, no index days
    df = patient_df.dropna(subset=["time"]).copy()
    if df.empty:
        return []
    df["time"] = pd.to_datetime(df["time"], errors="coerce")
    df = df.dropna(subset=["time"])

    # Work at day granularity
    df["day"] = df["time"].dt.normalize()

    # Build a set of all calendar days covered by any cluster (inclusive)
    cluster_day_set = {
        pd.Timestamp(d).normalize()
        for start_day, end_day, _ in clusters
        for d in pd.date_range(start_day, end_day, freq="D")
    }

    # Iterate through unique observed days in time order
    day_rows = df[["day"]].drop_duplicates().sort_values("day").reset_index(drop=True)

    index_list, prev_day = [], None
    sid = patient_df["subject_id"].iloc[0]

    for i in range(len(day_rows)):
        current_day = day_rows.loc[i, "day"]
        in_cluster = current_day in cluster_day_set
        gap_days = math.inf if prev_day is None else (current_day - prev_day).days

        # Select "index" days: not in any cluster and preceded by a sufficiently long quiet gap
        if (not in_cluster) and gap_days >= sparse_gap_days:
            # Use the earliest timestamp on that day as the index time
            first_time = df.loc[df["day"] == current_day, "time"].min()

            # Also capture the source file for that (day, first_time) record, if available 
            # This is what enables leave-one-CSV-out cross-validation at the index-row level.
            if "source_file" in df.columns:
                src_candidates = df.loc[(df["day"] == current_day) & (df["time"] == first_time), "source_file"]
                src_file = src_candidates.iloc[0] if len(src_candidates) > 0 else "unknown"
            else:
                src_file = "unknown"

            # Return tuple: (subject_id, index_time, index_day, source_file)  
            index_list.append((sid, first_time, current_day, src_file))  
        prev_day = current_day

    return index_list


# Builds features for ONE (patient, index_time) row using a lookback window.
# Features per code:
# - last: last observed numeric_value at or before index_time
# - days_since_last: days between index_time and that last observation
# - median_{lookback_days}d: median of values in the window
# - iqr_{lookback_days}d: IQR in the window (computed only when enough points)
# - count_{lookback_days}d: number of observed values in the window
#
# NEW vs older versions:
# - now includes median + IQR + count in addition to last/days_since_last. 
def build_features_for_index(patient_df, index_time, lookback_days):
    # Build feature dict for a single index time using measurement data in a lookback window
    result = {}
    meas_df = patient_df.copy()

    # Restrict to measurement table if present (keeps features consistent with cluster_from_table default)
    if "table" in meas_df.columns:
        meas_df = meas_df[meas_df["table"] == "measurement"].copy()

    lookback_start = index_time - pd.Timedelta(days=lookback_days)
    codes = [CODE_SBP, CODE_HR, CODE_GLUCOSE, CODE_HBA1C]

    for code_val in codes:
        # Prefix for feature names (make code safe for column names)
        col_prefix = code_val.replace("/", "_")

        # Subset to this code with valid timestamps
        code_df = meas_df[(meas_df["code"] == code_val) & (~meas_df["time"].isna())].copy()

        # "Last" value at or before index_time, plus days since it occurred
        last_df = code_df[code_df["time"] <= index_time].sort_values("time")
        if last_df.empty:
            last_val = np.nan
            last_days_since = np.nan
        else:
            last_row = last_df.iloc[-1]
            last_val = last_row["numeric_value"]
            last_days_since = (index_time - last_row["time"]).days

        # Lookback window stats within [lookback_start, index_time]
        window_df = code_df[(code_df["time"] >= lookback_start) & (code_df["time"] <= index_time)]
        window_vals = window_df["numeric_value"].dropna()

        # If no values in window, return NaNs and count=0
        if window_vals.empty:
            window_median = np.nan
            window_iqr = np.nan
            window_count = 0
        else:
            window_median = float(window_vals.median())
            window_count = int(len(window_vals))

            # IQR stability rule: compute only when there are enough points 
            # (Avoids noisy/meaningless IQR for tiny sample sizes)
            if window_count >= 4:
                q1 = float(window_vals.quantile(0.25))
                q3 = float(window_vals.quantile(0.75))
                window_iqr = q3 - q1
            else:
                window_iqr = np.nan

        # Store features for this code
        result[f"{col_prefix}_last"] = last_val
        result[f"{col_prefix}_days_since_last"] = last_days_since
        result[f"{col_prefix}_median_{lookback_days}d"] = window_median
        result[f"{col_prefix}_iqr_{lookback_days}d"] = window_iqr
        result[f"{col_prefix}_count_{lookback_days}d"] = window_count

    return result


# Assigns a binary label for an index timestamp:
# Positive (1) if ANY cluster starts within the prediction window after index_time.
# - Window start = index_time + (EXCLUSION_BEFORE_CLUSTER_DAYS + 1)
# - Window end   = index_time + PREDICTION_HORIZON_DAYS
# This avoids labeling clusters that begin immediately after the index day (exclusion buffer).
def label_index_row(index_time, clusters):
    # No clusters means no positive label
    if not clusters:
        return 0

    # Define the prediction window: start after an exclusion buffer, end at the prediction horizon
    horizon_start = (index_time + pd.Timedelta(days=EXCLUSION_BEFORE_CLUSTER_DAYS + 1)).normalize()
    horizon_end = (index_time + pd.Timedelta(days=PREDICTION_HORIZON_DAYS)).normalize()

    # Positive if any cluster starts within [horizon_start, horizon_end]
    for (start_time, end_time, n) in clusters:
        if (start_time >= horizon_start) and (start_time <= horizon_end):
            return 1

    # Otherwise negative
    return 0


def main():
    # CLI interface so you can run with different CSVs / parameters without editing the file.
    parser = argparse.ArgumentParser(description="Beginner temporal hospitalization model (multi-CSV)")
    parser.add_argument("-f", default=None, help=argparse.SUPPRESS)  # keeps compatibility with some notebook runners
    parser.add_argument("--csv", nargs="+", default=DEFAULT_CSV_PATHS,
                        help="One or more CSV file paths.")
    parser.add_argument("--test-size", type=float, default=DEFAULT_TEST_SIZE)
    parser.add_argument("--lookback-days", type=int, default=DEFAULT_LOOKBACK_DAYS)
    parser.add_argument("--sparse-gap-days", type=int, default=DEFAULT_SPARSE_GAP_DAYS)
    parser.add_argument("--dense-gap-days", type=int, default=DEFAULT_DENSE_GAP_DAYS)
    parser.add_argument("--min-cluster-days", type=int, default=DEFAULT_MIN_CLUSTER_DAYS)
    parser.add_argument("--cluster-from-table", default=DEFAULT_CLUSTER_FROM_TABLE)
    args, unknown = parser.parse_known_args()
    if unknown:
        # Ignore unknown args rather than failing
        print(f"Ignoring unknown args: {unknown}")

    # Allow --cluster-from-table none to disable the restriction
    cluster_from_table = None if str(args.cluster_from_table).lower() == "none" else args.cluster_from_table

    # Load all CSVs, tag each with source_file, then combine  
    dfs = []
    for path in args.csv:
        p = Path(path).expanduser()
        part = load_csv_simple(str(p))
        # Tag each row with the CSV filename it came from (enables leave-one-CSV-out CV)  
        part["source_file"] = p.name
        dfs.append(part)

    # Combine all patient events across files, keeping chronological order per subject
    df = (pd.concat(dfs, ignore_index=True)
            .sort_values(["subject_id", "time"])
            .reset_index(drop=True))
    print(f"Combined rows: {len(df)}  Files loaded: {len(dfs)}")

    subjects = df["subject_id"].dropna().unique()
    print(f"Unique subjects: {len(subjects)}")

    rows = []

    # Faster + avoids repeated df[df["subject_id"]==sid] scans by grouping once  
    for sid, patient in df.groupby("subject_id", sort=False):  
        # Find clusters (dense runs of observation days)
        clusters = find_clusters_for_patient(
            patient,
            sparse_gap_days=args.sparse_gap_days,
            dense_gap_days=args.dense_gap_days,
            min_cluster_days=args.min_cluster_days,
            cluster_from_table=cluster_from_table
        )

        # Build index days: non-cluster days after sparse gaps
        index_days = build_index_days(patient, clusters, sparse_gap_days=args.sparse_gap_days)

        # For each index day, compute features + label
        for sid2, idx_time, idx_day, src_file in index_days: 
            feats = build_features_for_index(patient, idx_time, lookback_days=args.lookback_days)
            label = label_index_row(idx_time, clusters)
            row = {
                "subject_id": sid2,
                "index_time": idx_time,
                "index_day": idx_day,
                "label": label,
                "source_file": src_file  
            }
            row.update(feats)
            rows.append(row)

    # Final modeling dataset: one row per (patient, index_time)
    data = (pd.DataFrame(rows)
              .sort_values(["subject_id", "index_time"])
              .reset_index(drop=True))

    print(f"Dataset size: {len(data)}  Positives: {data['label'].sum()}  Negatives: {len(data)-data['label'].sum()}")

    # Metadata columns are not used as ML features  
    meta_cols = ["subject_id", "index_time", "index_day", "label", "source_file"]  

    # All remaining columns are model features
    feature_cols = [c for c in data.columns if c not in meta_cols]

    # Identify distinct CSV sources (each one becomes a held-out fold)  
    files = sorted(data["source_file"].dropna().unique().tolist())  
    if len(files) < 2:
        print("Not enough distinct source_file values to run leave-one-CSV-out CV.")
        return

    # Accumulate out-of-fold labels and probabilities (so we report ONE combined score) 
    y_true_all = []
    y_prob_all = []

    for test_file in files:
        # Split by held-out CSV (test fold = one file, train fold = all other files)  
        test_df = data[data["source_file"] == test_file]
        train_df = data[data["source_file"] != test_file]

        # Skip degenerate folds (should rarely happen, but kept safe)
        if test_df.empty or train_df.empty:
            continue

        # Extract features and labels
        X_train = train_df[feature_cols].to_numpy(dtype=float)
        y_train = train_df["label"].astype(int).to_numpy()
        X_test = test_df[feature_cols].to_numpy(dtype=float)
        y_test = test_df["label"].astype(int).to_numpy()

        # Skip fold if training data has only one class (can't fit a classifier)
        if len(np.unique(y_train)) < 2:
            continue

        # Trees don't need scaling, but they DO need NaNs handled → median imputation
        # Fit imputer on train only to avoid test leakage
        imputer = SimpleImputer(strategy="median")
        X_train_imp = imputer.fit_transform(X_train)
        X_test_imp = imputer.transform(X_test)

        # Train balanced random forest (class_weight helps with imbalance)
        model = RandomForestClassifier(
            n_estimators=600,
            max_features="sqrt",
            min_samples_leaf=5,
            class_weight="balanced_subsample",
            random_state=42,
            n_jobs=-1
        )
        model.fit(X_train_imp, y_train)

        # Predict probabilities for the held-out CSV
        y_prob = model.predict_proba(X_test_imp)[:, 1]

        # Store out-of-fold predictions (pooled later)
        y_true_all.append(y_test)
        y_prob_all.append(y_prob)

    # Abort if no valid folds produced predictions
    if not y_true_all:
        print("No CV folds produced predictions (check class balance per fold / source_file assignment).")
        return

    # Pool all out-of-fold predictions into one evaluation
    y_true_all = np.concatenate(y_true_all)
    y_prob_all = np.concatenate(y_prob_all)
    y_pred_all = (y_prob_all >= 0.5).astype(int)  # default 0.5 threshold

    print(f"Test rows: {len(y_true_all)}  Positives: {int(y_true_all.sum())}  Negatives: {int(len(y_true_all)-y_true_all.sum())}")

    # Only compute AUROC/AUPRC if both classes exist in pooled test set
    if len(np.unique(y_true_all)) > 1:
        print("AUROC:", round(roc_auc_score(y_true_all, y_prob_all), 3))
        print("AUPRC:", round(average_precision_score(y_true_all, y_prob_all), 3))

    # Brier score measures calibration (lower is better)
    print("Brier:", round(brier_score_loss(y_true_all, y_prob_all), 4))

    # Classification report at threshold 0.5 (precision/recall/F1 per class)
    print(classification_report(y_true_all, y_pred_all, digits=3))


if __name__ == "__main__":
    main()


IT IS RUNNING!
Loading CSV from /Users/colleenohare/Desktop/Bioinformatics_MSC/RETFOUND/Chris_Sainsbury/Inspect_Dataset/inspect_data_csv/data_0.csv
Loading CSV from /Users/colleenohare/Desktop/Bioinformatics_MSC/RETFOUND/Chris_Sainsbury/Inspect_Dataset/inspect_data_csv/data_1.csv
Loading CSV from /Users/colleenohare/Desktop/Bioinformatics_MSC/RETFOUND/Chris_Sainsbury/Inspect_Dataset/inspect_data_csv/data_2.csv
Loading CSV from /Users/colleenohare/Desktop/Bioinformatics_MSC/RETFOUND/Chris_Sainsbury/Inspect_Dataset/inspect_data_csv/data_3.csv
Loading CSV from /Users/colleenohare/Desktop/Bioinformatics_MSC/RETFOUND/Chris_Sainsbury/Inspect_Dataset/inspect_data_csv/data_4.csv
Combined rows: 10502789  Files loaded: 5
Unique subjects: 946
Dataset size: 62171  Positives: 4058  Negatives: 58113
Test rows: 62171  Positives: 4058  Negatives: 58113
Prevalence: 0.0653
AUROC: 0.63
AUPRC: 0.118
Brier: 0.1939
Predicted prob quantiles [min,50%,90%,99%,max]: [0.0028 0.3974 0.6302 0.8022 0.9541]

=== Cla